In [11]:
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
import swifter

In [3]:
train_df = pd.read_csv('../data/train.csv.zip').dropna()
test_df = pd.read_csv('../data/test.csv.zip').dropna()
train_df.head()

/home/omid-dev/Personal/venvs/kaggle/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
train_df['simple_ratio'] = train_df.swifter.apply(lambda x: fuzz.ratio(x['question1'], x['question2']), axis=1)
train_df['partial_ratio'] = train_df.swifter.apply(lambda x: fuzz.partial_ratio(x['question1'], x['question2']), axis=1)
train_df['token_sort_ratio'] = train_df.swifter.apply(lambda x: fuzz.token_sort_ratio(x['question1'], x['question2']), axis=1)
train_df['token_set_ratio'] = train_df.swifter.apply(lambda x: fuzz.token_set_ratio(x['question1'], x['question2']), axis=1)

Dask Apply: 100%|██████████| 32/32 [00:16<00:00,  1.93it/s]


,id,qid1,qid2,question1,question2,is_duplicate,simple_ratio,partial_ratio,token_sort_ratio,token_set_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,93,98,93,100
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,65,73,63,86
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,45,41,63,63
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,7,20,24,28
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,37,54,47,67
